In [ ]:
# import sys
# !{sys.executable} -m pip install  mysql.connector
import csv
import pandas as pd
import csv
from sqlalchemy import create_engine
import mysql.connector
import numpy as np
from datetime import datetime

In [ ]:
 import sys
 print(sys.version)

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [ ]:
# Load the pkl file of inspection/crash data combined for 2020
crash_inspect_20 = pd.read_pickle('/home/propel/FMCSA/MCMIS/crash_inspect.pkl')
crash_inspect_20['DOT_NUMBER'].fillna(0,inplace=True)
crash_inspect_20['DOT_NUMBER'] = crash_inspect_20['DOT_NUMBER'].round(0).astype(int)
crash_inspect_20.shape

(1675966, 6)

In [ ]:
# Load the file of inspection/crash data combined for 2019
crash_inspect_19 = pd.read_csv("/home/propel/FMCSA/MCMIS/crash_inspect_19.csv")
crash_inspect_19['DOT_NUMBER'].fillna(0,inplace=True)
crash_inspect_19['DOT_NUMBER'] = crash_inspect_19['DOT_NUMBER'].round(0).astype(int)
crash_inspect_19.shape

(3588674, 7)

In [ ]:
# Merge 19 and 20
crash_insp_20_19 = pd.merge(crash_inspect_20,crash_inspect_19, how='outer')   
crash_insp_20_19.head(1)

,ID,INCIDENT_DATE,DOT_NUMBER,VEHICLE_ID_NO,LICENSE_PLATE_NO,INCIDENT,Unnamed: 0
0,3932969,2020-02-11,13,1XP5DB9X91N550659,00136V,CRASH,NaN


In [ ]:
# Load state/carrier data 20/19
crash_inspect_st_1920 = pd.read_csv('/home/propel/FMCSA/MCMIS/crash_inspect_st_1920.csv')
crash_inspect_st_1920.shape

(3216631, 4)

In [ ]:
# Merge crash/inspection with city/name data
crash_insp = pd.merge(crash_insp_20_19, crash_inspect_st_1920, how='left', on=['ID'])
crash_insp.shape

(5264640, 10)

In [ ]:
#crash_insp_20_19.head(3)
crash_insp = crash_insp.iloc[:,[1,2,3,5,8,9]]

crash_insp['Count'] = crash_insp.groupby(['DOT_NUMBER','VEHICLE_ID_NO'])['INCIDENT_DATE'].transform('count')
crash_insp.drop_duplicates()

,INCIDENT_DATE,DOT_NUMBER,VEHICLE_ID_NO,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
0,2020-02-11,13,1XP5DB9X91N550659,CRASH,CORBIN CLAUDE E,GA,1.0
1,2020-01-18,198,1FUJGLDR7DSBW7567,CRASH,TRANSPORTATION SERVICES INC,MI,1.0
2,2020-02-27,198,3HSDZTZR7LN884788,CRASH,TRANSPORTATION SERVICES INC,MI,1.0
3,2020-03-07,198,3AKJGLDR6KDKK6282,CRASH,T S I,MI,1.0
4,2020-03-20,198,3HSDZTZR8KN126523,CRASH,TRANSPORTATION SERVICES INC,MI,1.0
...,...,...,...,...,...,...,...
5264635,2019-08-07,3007911,3FRWX6FL5FV727647,INSPECTION,ANTHONY'S TOWING AND TRANSPORTATION,CA,3.0
5264636,2019-08-07,908846,4V4NC9EH5LN226417,INSPECTION,ROADRUNNER TEMPERATURE CONTROLLED LLC,NE,2.0
5264637,2019-08-07,908846,1UYVS2533DM713215,INSPECTION,ROADRUNNER TEMPERATURE CONTROLLED LLC,NE,2.0
5264638,2019-08-02,74432,3AKJGLDR0JSJH6520,INSPECTION,MARTEN TRANSPORT LTD,WI,6.0


In [ ]:
# Check for duplicated vehicle ID number
ownersh_20 = crash_insp
own_change = ownersh_20[ownersh_20['Count'] > 1] 
# Check for duplicated vehicle ID number
own_change = own_change[own_change.duplicated(['VEHICLE_ID_NO'])]

# # remove incidents without DOT numbers
own_change = own_change[own_change['DOT_NUMBER'] > 0]

own_change =  own_change.sort_values(by=['VEHICLE_ID_NO'])
own_change

,INCIDENT_DATE,DOT_NUMBER,VEHICLE_ID_NO,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
2622489,2019-02-28,265752,***,INSPECTION,FEDEX GROUND PACKAGE SYSTEM INC,PA,2.0
906490,2020-02-28,2598416,****,INSPECTION,SHARP EXPRESS INC,ON,2.0
1357087,2020-04-21,121058,",",INSPECTION,UPS FREIGHT,VA,3.0
3723904,2019-05-09,252802,",",INSPECTION,TRANSPORT BOURASSA INC,PQ,5.0
1878496,2019-01-09,558713,",",INSPECTION,NORMANDIN TRANSIT INC,PQ,51.0
...,...,...,...,...,...,...,...
4099268,2019-06-03,964715,ZA96A4J1231A24359,INSPECTION,MID-AMERICA SHOWS TRANSPORTATION INC,IN,2.0
4792937,2019-07-12,1314372,ZMN01JAH051007208,INSPECTION,DAVID WEETS TRUCKING INC,MN,2.0
4804351,2019-07-12,2753843,ZP28657,INSPECTION,SANCON ENGINEERING INC,CA,2.0
5191973,2019-08-06,2403113,ZT1147,INSPECTION,DREW HUTCHENS TRUCKING,AR,2.0


In [ ]:
# Remove non alphanumeric from vehicle_ID_NO
crash_insp_20_19 = own_change[own_change.VEHICLE_ID_NO.str.isalnum()]
crash_insp_20_19

,INCIDENT_DATE,DOT_NUMBER,VEHICLE_ID_NO,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
711838,2020-02-09,707214,0,INSPECTION,COUTURE EXPRESSWAY,PQ,5.0
1640588,2020-06-02,265752,0,INSPECTION,FEDEX GROUND PACKAGE SYSTEM INC,PA,9.0
1189372,2020-03-20,1486168,0,INSPECTION,NFI TRANSPORTATION,NJ,2.0
711955,2020-01-27,57911,0,INSPECTION,BOZZUTOS INC,CT,3.0
1541728,2020-05-22,2146709,0,INSPECTION,MONTGOMERY TRANSPORT LLC,AL,3.0
...,...,...,...,...,...,...,...
3370962,2019-04-18,2563035,Z90CVC12851,INSPECTION,JUAN NABOR AGUILERA SUAREZ,TX,3.0
4099268,2019-06-03,964715,ZA96A4J1231A24359,INSPECTION,MID-AMERICA SHOWS TRANSPORTATION INC,IN,2.0
4792937,2019-07-12,1314372,ZMN01JAH051007208,INSPECTION,DAVID WEETS TRUCKING INC,MN,2.0
4804351,2019-07-12,2753843,ZP28657,INSPECTION,SANCON ENGINEERING INC,CA,2.0


In [ ]:
# crash_insp_20_19 = crash_insp_20_19.iloc[:,[0,1,2]]
crash_insp_20_19['INCIDENT_DATE'] = pd.to_datetime(crash_insp_20_19['INCIDENT_DATE']).dt.strftime('%Y-%m-%d')

,INCIDENT_DATE,USDOT,VEHICLE_ID_NO,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
711838,2020-02-09,707214,0,INSPECTION,COUTURE EXPRESSWAY,PQ,5.0
1640588,2020-06-02,265752,0,INSPECTION,FEDEX GROUND PACKAGE SYSTEM INC,PA,9.0
1189372,2020-03-20,1486168,0,INSPECTION,NFI TRANSPORTATION,NJ,2.0
711955,2020-01-27,57911,0,INSPECTION,BOZZUTOS INC,CT,3.0
1541728,2020-05-22,2146709,0,INSPECTION,MONTGOMERY TRANSPORT LLC,AL,3.0
...,...,...,...,...,...,...,...
3370962,2019-04-18,2563035,Z90CVC12851,INSPECTION,JUAN NABOR AGUILERA SUAREZ,TX,3.0
4099268,2019-06-03,964715,ZA96A4J1231A24359,INSPECTION,MID-AMERICA SHOWS TRANSPORTATION INC,IN,2.0
4792937,2019-07-12,1314372,ZMN01JAH051007208,INSPECTION,DAVID WEETS TRUCKING INC,MN,2.0
4804351,2019-07-12,2753843,ZP28657,INSPECTION,SANCON ENGINEERING INC,CA,2.0


In [ ]:
group = crash_insp_20_19.groupby('VEHICLE_ID_NO')

# Lambda makes columns of unique 'USDOT' numbers per 'Vehicle_ID_NO'
df2 = group.apply(lambda x: x['DOT_NUMBER'].unique())

df2 = df2.apply(pd.Series)

df2 = df2[df2[1].notna()]
df2

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
VEHICLE_ID_NO,,,,,,,,,,,,,,,,,,,,,
0,707214.0,265752.0,1486168.0,57911.0,2146709.0,89243.0,264184.0,165420.0,239039.0,241829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000,46749.0,241829.0,2541970.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00000000000000000,71821.0,265752.0,16130.0,2939491.0,239039.0,1254531.0,62227.0,2968731.0,73705.0,3706.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000171774MO,3025502.0,3310239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0014007A07074129,2273644.0,2902899.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YE2CC16B782046787,707525.0,1975840.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YE2CC22B122045783,2372753.0,3293105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YE2XB83B5G3048791,905572.0,943538.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Reset index
reset_ind = df2.reset_index()
#Drop Unknown
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'UNKNOWN']
#Drop UNKOWN
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'UNKOWN']
#Drop None
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'NONE']
#Drop UNK
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'UNK']
#Drop UKN
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'UKN']
#Remove 00000000000000000 vehicle ID
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != '00000000000000000']
# Remove Zero vehicle ID
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != '0']
# Remove X vehicle ID
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'X']

reset_ind.rename(columns={0:"DOT_NO1",1:"DOT_NO2",2:"DOT_NO3",3:"DOT_NO4",4:"DOT_NO5",5:"DOT_NO6",6:"DOT_NO7"},inplace=True)
reset_ind.dropna(axis=1,how='all',inplace=True)
reset_ind

,VEHICLE_ID_NO,DOT_NO1,DOT_NO2,DOT_NO3,DOT_NO4,DOT_NO5,DOT_NO6,DOT_NO7
1,0000,46749.0,241829.0,2541970.0,NaN,NaN,NaN,NaN
3,000171774MO,3025502.0,3310239.0,NaN,NaN,NaN,NaN,NaN
4,0014007A07074129,2273644.0,2902899.0,NaN,NaN,NaN,NaN,NaN
5,0018,2712352.0,59659.0,NaN,NaN,NaN,NaN,NaN
6,0HT003101,3067919.0,1582105.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
14538,YE2CC16B782046787,707525.0,1975840.0,NaN,NaN,NaN,NaN,NaN
14539,YE2CC22B122045783,2372753.0,3293105.0,NaN,NaN,NaN,NaN,NaN
14540,YE2XB83B5G3048791,905572.0,943538.0,NaN,NaN,NaN,NaN,NaN
14541,YE2XC81B4H3049139,397239.0,3271206.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
######### OOS Carriers ############

In [ ]:
OOS_data = pd.read_csv("/home/propel/FMCSA/MCMIS/CL_OS_Coords.csv")
OOS_sub = OOS_data.loc[:,['USDOT','OOS_DATE','DBA_NAME','STATE']]
OOS_sub['OOS_DATE'] = pd.to_datetime(OOS_sub['OOS_DATE']).dt.strftime('%Y-%m-%d')
OOS_sub.head(3)

,USDOT,OOS_DATE,DBA_NAME,STATE
0,1180829,2004-10-04,C & S TRUCKING,AL
1,1189597,2004-09-14,HARLIN TRUCKING,AL
2,1194141,2004-09-14,,AL


In [ ]:
# Rename to USDOT
reset_ind1 = reset_ind.iloc[:,[0,1]].rename(columns={"DOT_NO1":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result1 = pd.merge(reset_ind1, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident1 =  result1.sort_values(by=['VEHICLE_ID_NO'])

Incident1.dropna(axis=0,how='any',inplace=True)
Incident1.shape

(40443, 7)

In [ ]:
# Rename to USDOT
reset_ind2 = reset_ind.iloc[:,[0,2]].rename(columns={"DOT_NO2":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result2 = pd.merge(reset_ind2, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident2 =  result2.sort_values(by=['VEHICLE_ID_NO'])

Incident2.dropna(axis=0,how='any',inplace=True)
Incident2.shape

(29062, 7)

In [ ]:
# Rename to USDOT
reset_ind3 = reset_ind.iloc[:,[0,3]].rename(columns={"DOT_NO3":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result3 = pd.merge(reset_ind3, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident3 =  result3.sort_values(by=['VEHICLE_ID_NO'])

Incident3.dropna(axis=0,how='any',inplace=True)
Incident3.shape

(763, 7)

In [ ]:
# Rename to USDOT
reset_ind4 = reset_ind.iloc[:,[0,4]].rename(columns={"DOT_NO4":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result4 = pd.merge(reset_ind4, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident4 =  result4.sort_values(by=['VEHICLE_ID_NO'])

Incident4.dropna(axis=0,how='any',inplace=True)
Incident4.shape

(59, 7)

In [ ]:
reset_ind5 = reset_ind.iloc[:,[0,5]].rename(columns={"DOT_NO5":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result5 = pd.merge(reset_ind5, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident5 =  result5.sort_values(by=['VEHICLE_ID_NO'])

Incident5.dropna(axis=0,how='any',inplace=True)
Incident5.shape

(14, 7)

In [ ]:
reset_ind6 = reset_ind.iloc[:,[0,6]].rename(columns={"DOT_NO6":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result6 = pd.merge(reset_ind6, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident6 =  result6.sort_values(by=['VEHICLE_ID_NO'])

Incident6.dropna(axis=0,how='any',inplace=True)
Incident6.shape

(8, 7)

In [ ]:
reset_ind7 = reset_ind.iloc[:,[0,7]].rename(columns={"DOT_NO7":"USDOT"})

# Rename for merging from
crash_insp_20_19 = crash_insp_20_19.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result7 = pd.merge(reset_ind7, crash_insp_20_19,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident7 =  result7.sort_values(by=['VEHICLE_ID_NO'])

Incident7.dropna(axis=0,how='any',inplace=True)
Incident7.shape

(3, 7)

In [ ]:
# Concat all 
Incidents_19_20 = pd.concat([Incident1,Incident2,Incident3,Incident4,Incident5,Incident6,Incident7],axis = 0)
Incidents_19_20.shape

(70352, 7)

In [ ]:
# Some formatting steps
Incidents_19_20 =  Incidents_19_20.sort_values(by=['VEHICLE_ID_NO','INCIDENT_DATE'])
Incidents_19_20['USDOT'] = Incidents_19_20['USDOT'].round(0).astype(int)
Incidents_19_20.reset_index(inplace=True)
Incidents_19_20.drop(['index'],axis=1,inplace=True)

In [ ]:
# Vehilces with multiple USDOTs
Mult_USDOT_19_20 = Incidents_19_20.groupby('VEHICLE_ID_NO').filter(lambda g: len(g) > 1).drop_duplicates(subset=['VEHICLE_ID_NO','USDOT'], keep="first")

In [ ]:
Mult_USDOT_19_20.head(10)

,VEHICLE_ID_NO,USDOT,INCIDENT_DATE,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
0,0000,241829,2019-07-04,INSPECTION,XPO LOGISTICS FREIGHT INC,MI,3.0
2,0000,2541970,2020-03-05,INSPECTION,JR TRANSPORTATION INC,WA,2.0
4,0000,46749,2020-04-09,INSPECTION,MILLIS TRANSFER LLC,WI,2.0
7,000171774MO,3025502,2019-03-14,INSPECTION,MAGEE AUTO TRANSPORT LIMITED LIABILITY COMPANY,MS,2.0
9,000171774MO,3310239,2020-02-10,INSPECTION,SMART CHOICE TRANSPORT LLC,MS,3.0
11,0014007A07074129,2273644,2019-01-17,INSPECTION,ADRIAN EDMUNDO CORDOVA BERNAL,TX,6.0
15,0014007A07074129,2902899,2019-03-08,INSPECTION,RODOLFO FLORES RODRIGUEZ,TX,4.0
20,0018,2712352,2019-01-14,INSPECTION,CHRISTOPHER VALENZA,CA,2.0
21,0018,59659,2019-06-03,INSPECTION,SOUTH COAST LUMBER CO,OR,2.0
23,0HT003101,3067919,2019-01-10,INSPECTION,SITA,CI,6.0


In [ ]:
# # OOS carriers merged with above on USDOT
# OOS_19_20 = pd.merge(OOS_sub,Mult_USDOT_19_20,on=['USDOT'])
# OOS_19_20 =  OOS_19_20.sort_values(by=['CARRIER_NAME'])
# OOS_19_20.tail(1)  

,USDOT,OOS_DATE,DBA_NAME,STATE,VEHICLE_ID_NO,INCIDENT_DATE,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
151,2991853,2020-01-08,,CA,1FUJGLD51GLHL5226,2019-03-26,INSPECTION,ZRT EXPRESS,CA,2.0


In [ ]:
# # Carriers who had incidents after being OOS
# Indt_after_19_20 = OOS_19_20.loc[OOS_19_20['OOS_DATE'] < OOS_19_20['INCIDENT_DATE']]
# Indt_after_19_20 =  Indt_after_19_20.sort_values(by=['CARRIER_NAME'])
# Indt_after_19_20.tail(10)

,USDOT,OOS_DATE,DBA_NAME,STATE,VEHICLE_ID_NO,INCIDENT_DATE,INCIDENT,CARRIER_NAME,CARRIER_STATE,Count
368,3286403,2020-01-04,,FL,5PVNE8JV8H4S55449,2020-02-10,INSPECTION,PAUL DELIVERY AND LOGISTICS LLC,FL,2.0
644,3113551,2018-09-17,,MD,1FD0W5HT1JEB96251,2019-04-09,INSPECTION,SB TRANSPORTATION LLC,MD,2.0
229,2564654,2018-03-26,SMARTWAY TRANSPORT,CA,3AKJGLD54GSGS5937,2020-02-01,INSPECTION,SMARTWAY DISTRIBUTION SERVICE INC,CA,3.0
201,1948453,2011-05-05,SADAF FOODS,CA,3AKJGEDV3HSJA9803,2019-02-28,INSPECTION,SOOFER CO INC,CA,3.0
200,1948453,2011-05-05,SADAF FOODS,CA,1UYVS2366H2088601,2019-05-31,INSPECTION,SOOFER CO INC,CA,2.0
640,2504063,2015-01-06,,LA,4V4NC9EH8EN165700,2019-03-07,INSPECTION,TAINO TRUCKING LLC,LA,2.0
857,3226400,2019-04-01,,TX,1KKVD5221YL201430,2019-04-04,INSPECTION,THE PRIVATEER JOURNEY LLC,TX,2.0
851,3232582,2019-12-23,,TX,2HSFBAMR1SC015404,2020-05-14,INSPECTION,TOUCH DOWN TRUCKING LLC,TX,2.0
596,3179608,2019-03-26,,IL,1XPBD49X1GD336553,2019-03-27,INSPECTION,VGV SERVICES INC,IL,2.0
634,2846409,2019-01-22,,KY,2HSFHASR5WC052589,2019-02-22,INSPECTION,WAYNE BASTIN TRUCKING LLC,KY,2.0


In [ ]:
# Save the 19_20 Incidents the files
# Mult_USDOT_19_20.to_csv('/home/propel/FMCSA/MCMIS/2019_2020_results/Mult_USDOT_19_20.csv')

In [ ]:
############ Connect to MySQl ###########

In [ ]:
try:
  engine = create_engine('mysql+mysqlconnector://routerisk:**********@*********/route_risk') #trip_time_efficiency
except engine.closed():
    print("Failed to create engine")

In [ ]:
try:   
  # Connect to DB- mysql.connector
  connection = mysql.connector.connect(
              user='routerisk',
              password='************',
              host='************',
              database='route_risk',
              use_pure=True
              )
  # Create cursor
  cursor = connection.cursor(prepared=True)
except mysql.connector.Error as error:
    print("Failed to execute stored procedure: {}".format(error))
  
finally:
    if (connection.is_connected()):
        cursor.close()
        print("Connection Established")

Connection Established


In [ ]:
# Save the Mult_USDOT_19_20 data to the route_risk database
Mult_USDOT_19_20.to_sql('Mult_USDOT_19_20',engine,if_exists ='replace',index=False)

In [ ]:
# Verify a successful save by reading file and counting the rows
cursor = connection.cursor(prepared=True)
sql_select_Query = "select * from Mult_USDOT_19_20"
cursor.execute(sql_select_Query)
records = cursor.fetchall()
print("Total number of rows in Mult_USDOT_19_20 is: ", cursor.rowcount)

Total number of rows in Mult_USDOT_19_20 is:  29494
